In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow import set_random_seed
from numpy.random import seed
from datetime import datetime

# Disable those annoying warnings
tf.logging.set_verbosity(tf.logging.ERROR)
# Set numpy seed
seed(1)
# Set tf seed
set_random_seed(2)
#
# Turn off GPU usage for tf
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
#

In [ ]:
# Read the previously saved dataframe
df_covtype_ohe = pd.read_csv(os.path.join(os.getcwd(), 'covtype_ohe.csv'), dtype=np.float32)

In [ ]:
def train_test_split(df_covtype_ohe):
    '''
    Split the one hot encoded dataset onto training set and test set
    according to UCI's repository guidelines
    '''
    # First 15120 rows for the training set
    X_train = df_covtype_ohe[:15120].copy()
    # The last seven colums are the targets
    X_train, y_train = X_train.iloc[:, :51], X_train.iloc[:, 51:]
    # The remaining rows are for the test set
    X_test = df_covtype_ohe[15120:].copy()
    X_test, y_test = X_test.iloc[:, :51], X_test.iloc[:, 51:]
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(df_covtype_ohe)

In [ ]:
# Check shapes consistency
print(f'X_train: {X_train.shape}, X_test: {X_test.shape}, ' \
      f'y_train: {y_train.shape}, y_test: {y_test.shape}')

In [ ]:
# Let's standardize the training set and test set.
# Note that we use the training set ONLY to calculate the mean and standard deviation
# then normalize the training set 
# and finally use the (training) mean and standard deviation to normalize the test set.
# This ensures no data leakage.

def train_test_normalize(X_train, X_test):
    '''
    Perform standardization on the training set and transforms the
    test set accordingly
    '''
    # The numerical columns we want to normalize
    numerical_columns = ['Elevation',
                         'Distance_To_Hydrology',
                         'Horizontal_Distance_To_Roadways',
                         'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
                         'Horizontal_Distance_To_Fire_Points']
    # Calculate the mean and standard deviation of the training set
    X_train_num_cols_mean = X_train[numerical_columns].mean()
    X_train_num_cols_std = X_train[numerical_columns].std()
    # Perform standardization over the numerical columns of the training set
    X_train_std = (X_train[numerical_columns] - X_train_num_cols_mean) / X_train_num_cols_std
    # Concatenate side-by-side the normalized training set and the one-hot encoded features
    # Note that we index X_train dataframe by the (set) difference of the overall features
    # minus the numerical ones
    ohe_features = X_train.columns.difference(other=numerical_columns, sort=False)
    X_train_std = pd.concat([X_train_std, X_train[ohe_features]], axis=1)
    # Perform standardization over the numerical columns of the test set, using the mean and std
    # of the training set as discussed earlier
    X_test_std = (X_test[numerical_columns] - X_train_num_cols_mean) / X_train_num_cols_std
    # Concatenate side-by-side the normalized test set and the one-hot encoded features
    X_test_std = pd.concat([X_test_std, X_test[ohe_features]], axis=1)
    return X_train_std, X_test_std

X_train_std, X_test_std = train_test_normalize(X_train, X_test)

In [ ]:
# Convert train/test sets to numpy float32 ndarrays
# (after doing some research it appears that float32 should be used when using a GPU)
X_train_std = X_train_std.to_numpy().astype(np.float32)
X_test_std = X_test_std.to_numpy().astype(np.float32)
y_train = y_train.to_numpy().astype(np.float32)
y_test = y_test.to_numpy().astype(np.float32)

In [ ]:
# Initialize the training/validation set splitter.
# We'll use it later for the grid-search, performing a 10-fold cross validation
# where each validation set is 25% the size of the training set.
# This yields 1620 samples per class for the new training set and
# 540 samples per class for each validation set.
validation_strat = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=10)

In [ ]:
# Subclass KerasClassifier and override fit() method
# to fix OOM error when running sklearn GridSearchCV on the GPU
class GridKerasClassifier(keras.wrappers.scikit_learn.KerasClassifier):
    def fit(self, *args, **kwargs):
        # Clear tensorflow session each time fit is invoked
        keras.backend.clear_session()
        # Use custom configuration for the new session
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        session = tf.Session(config=config)
        keras.backend.set_session(session)
        super().fit(*args, **kwargs)

In [ ]:
# Wrapper function which builds the classifier architecture of the baseline model.
def baseline_model(hidden_layers=0, h1_size=120, h2_size=0,
                   n_features=51, n_classes=7, learning_rate=0.5):
    model = keras.Sequential()
    model.add(keras.layers.Dense(h1_size, activation='sigmoid', input_dim=n_features))
    model.add(keras.layers.Dense(n_classes, activation='sigmoid'))
    sgd = keras.optimizers.SGD(lr=learning_rate)
    model.compile(optimizer=sgd, loss='mean_squared_error', metrics=['accuracy'])
    return model

In [ ]:
# Wrapper function which builds the classifier architecture of the model
# similar to the baseline one.
def crossentropy_model(hidden_layers=0, h1_size=120, h2_size=0,
                       n_features=51, n_classes=7, learning_rate=0.5):
    model = keras.Sequential()
    model.add(keras.layers.Dense(h1_size, activation='relu', input_dim=n_features))
    model.add(keras.layers.Dense(n_classes, activation='softmax'))
    sgd = keras.optimizers.SGD(lr=learning_rate)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Wrapper function which builds the classifier architecture of a two hidden layer model
def two_hidden_model(hidden_layers=2, h1_size=30, h2_size=15,
                       n_features=51, n_classes=7, learning_rate=0.3):
    model = keras.Sequential()
    model.add(keras.layers.Dense(h1_size, activation='relu', input_dim=n_features))
    model.add(keras.layers.Dense(h2_size, activation='relu'))
    model.add(keras.layers.Dense(n_classes, activation='softmax'))
    sgd = keras.optimizers.SGD(lr=learning_rate)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def bottleneck_model(hidden_layers=3, h1_size=46, h2_size=30, h3_size=15,
                      n_features=51, n_classes=7, learning_rate=0.3):
    model = keras.Sequential()
    # autoencoder component begins here
    model.add(keras.layers.Dense(h1_size, activation='relu', input_dim=n_features))
    model.add(keras.layers.Dense(n_features, activation='relu'))
    # autoencoder component ends here
    # Adds the two_hidden network right after the autoencoder
    model.add(keras.layers.Dense(h2_size, activation='relu'))
    model.add(keras.layers.Dense(h3_size, activation='relu'))
    model.add(keras.layers.Dense(n_classes, activation='softmax'))
    sgd = keras.optimizers.SGD(lr=learning_rate)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Wrap the Keras neural network onto a sklearn Classifier
net_to_gs = GridKerasClassifier(multi_hidden_model)

# For params tuning see
# https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

# Define the grid search params
# uncomment for changing the default number of features and/or classes
# for the network. DON'T FORGET TO CHANGE THE TRAINING/TEST SET ACCORDINGLY!
# n_features = [X_train_std.shape[1]]
# n_classes = [y_train.shape[1]]
hidden_layers = [1]
h1_size = [30]
h2_size = [15]
learning_rate = [0.3]
batch_size = [128]
epochs = [50, 100]
# Define the grid of parameters 
param_grid = dict(# n_features=n_features, # uncomment if needed
                  # n_classes=n_classes,   # uncomment if needed
                  hidden_layers=hidden_layers,
                  h1_size=h1_size,
                  h2_size=h2_size,
                  learning_rate=learning_rate,
                  batch_size=batch_size,
                  epochs=epochs)
# Initialize the grid search using the nn classifier and the cross-validation
# strategy defined above. Since clear_session() is invoked after each CV run,
# it's fine to set n_jobs=-1 when running on the GPU.
grid = GridSearchCV(estimator=net_to_gs,
                    param_grid=param_grid,
                    n_jobs=-1,
                    cv=validation_strat,
                    verbose=0)

In [ ]:
# Run grid search
# THIS CELL IS TIME CONSUMING, BE AWARE OF THAT!
grid.fit(X_train_std, y_train)
# Store the best model in the current dir
#now = datetime.now()
#timestamp = now.strftime('%b-%d-%Y_%H%M')
#best_gr_model_params = grid.best_params_
#best_gr_model_hlyrs = best_gr_model_params['hidden_layers']
#best_gr_model_h1size = best_gr_model_params['h1_size']
#best_gr_model_h2size = best_gr_model_params['h2_size']
#best_gr_model_epochs = best_gr_model_params['epochs']
#best_gr_model_lr = best_gr_model_params['learning_rate']
#best_gr_model_bs = best_gr_model_params['batch_size']
#best_model_fname = f'hlyrs{best_gr_model_hlyrs}_h1size{best_gr_model_h1size}_h2size{best_gr_model_h2size}' \
#                   f'_epochs{best_gr_model_epochs}_learning_r{best_gr_model_lr}_bsize{best_gr_model_bs}_{timestamp}.hdf5'
#best_model_path = os.path.join(os.getcwd(), best_model_fname)
#keras.models.save_model(model=grid.best_estimator_.model,
#                        filepath=best_model_path,
#                        save_format='h5')

In [ ]:
# Print the best scores defined earlier for the grid search and its model params
print(f'Best scores: {grid.best_score_:.4f} using {grid.best_params_}', end='\n\n')
# Get the mean of each score for each cross-validation run
means = grid.cv_results_['mean_test_score']
# Get the standard dev of each score for each cross-validation run
stds = grid.cv_results_['std_test_score']
# Get the model params for each cross-validation run
params = grid.cv_results_['params']
# Loop through means, stds, params and print
# one line for each cross-validation run
for mean, stdev, param in zip(means, stds, params):
    print(f'Mean: {mean:.4f} +- {stdev:.6f} with: {param}')

In [ ]:
# Do some cleanup
del grid
del net_to_gs
keras.backend.clear_session()

In [ ]:
# Run this cell if you already have a trained model
#model_filename = 'hlyrs2_h1size30_h2size15_epochs200_bsize128_Jul-15-2019_0932.hdf5'
#model_to_load_path = os.path.join(os.getcwd(), model_filename)
#loaded_model = tf.keras.models.load_model(model_to_load_path)
# Uncomment this block to plot a diagram of the model
#model_filename_no_ext = os.path.splitext(model_filename)[0]
#model_plot_filename = f'{model_filename_no_ext}.png'
#keras.utils.plot_model(loaded_model, to_file=model_plot_filename, show_shapes=True, show_layer_names=True)

In [ ]:
# Now that the grid search is done
# let's retrain the best model via 10-fold cross validation.

# Using plain KerasClassifier since our subclass failed returning a History object
selected_model = bottleneck_model() # call the desired model func() 
cv_train_acc = []
cv_val_acc = []
cv_train_loss = []
cv_val_loss = []
best_epochs = 100
best_batch_size = 128
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=1)
cv_fold_counter = 0
for train_index, valid_index in sss.split(X_train_std, y_train):
    X_train_std_minus_validation = X_train_std[train_index]
    X_validation = X_train_std[valid_index]
    y_train_minus_validation = y_train[train_index]
    y_validation = y_train[valid_index]
    history = selected_model.fit(X_train_std_minus_validation,
                                 y_train_minus_validation,
                                 epochs=best_epochs,
                                 batch_size=best_batch_size,
                                 validation_data=(X_validation, y_validation),
                                 verbose=0)
    cv_train_acc.append(history.history['acc'])
    cv_val_acc.append(history.history['val_acc'])
    cv_train_loss.append(history.history['loss'])
    cv_val_loss.append(history.history['val_loss'])
    cv_fold_counter += 1
    print(f'{cv_fold_counter}-fold completed.')

In [ ]:
# For each epoch, k=10 observations per metric are computed.
# For example, cv_train_acc shape is (n_kfolds, n_epochs).
# Thus, flatten acc/loss arrays and strip some values for plotting them later
# First the acc arrays
cv_train_acc_arr = np.array(cv_train_acc).flatten()
cv_val_acc_arr = np.array(cv_val_acc).flatten()
cv_train_acc_arr = cv_train_acc_arr[0:10*best_epochs:best_epochs]
cv_val_acc_arr = cv_val_acc_arr[0:10*best_epochs:best_epochs]
# Then the loss arrays
cv_train_loss_arr = np.array(cv_train_loss).flatten()
cv_val_loss_arr = np.array(cv_val_loss).flatten()
cv_train_loss_arr = cv_train_loss_arr[0:10*best_epochs:best_epochs]
cv_val_loss_arr = cv_val_loss_arr[0:10*best_epochs:best_epochs]

In [ ]:
# Plot training/validation accuracy vs number of folds for the 10-fold cross validated model
plt.figure()
plt.plot(range(1, cv_train_acc_arr.shape[0] + 1), cv_train_acc_arr, 'bo', label='Training acc')
plt.plot(range(1, cv_val_acc_arr.shape[0] + 1), cv_val_acc_arr, 'b', label='Validation acc')
plt.ylim(0.5, 1)
plt.title('Two hidden model train/val accuracy - 10-fold cv')
plt.ylabel('Accuracy')
plt.xlabel('Number of folds')
plt.grid()
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Plot training/validation loss vs number of folds for the 10-fold cross validated model
plt.figure()
plt.plot(range(1, cv_train_loss_arr.shape[0] + 1), cv_train_loss_arr, 'bo', label='Training loss')
plt.plot(range(1, cv_val_loss_arr.shape[0] + 1), cv_val_loss_arr, 'b', label='Validation loss')
plt.title('Two hidden model train/val loss - 10-fold cv')
plt.ylabel('Loss')
plt.xlabel('Number of folds')
plt.grid()
plt.legend(loc='upper right')
plt.show()

In [ ]:
# When we are satisfied about the best model run predict on the test set.
y_pred = selected_model.predict(X_test_std)
# Reverse one-hot encoding (i.e going back to categorical variables) for the predicted targets.
y_pred_cat = np.argmax(y_pred, axis=1)
# Do the same for the true targets
y_true_cat = np.argmax(y_test, axis=1)

In [ ]:
# Now that we have y_pred and y_true, we can use sklearn.metrics for precision, recall and f1-score metrics
target_names = ['Spruce/Fir', 'Lodgepole Pine',
                'Ponderosa Pine', 'Cottonwood/Willow',
                'Aspen', 'Douglas-fir', 'Krummholz']
print(classification_report(y_true_cat, y_pred_cat, target_names=target_names))

# Uncomment this block to save the classification report to a csv file.
#report = classification_report(y_true_cat, y_pred_cat, target_names=target_names, output_dict=True)
#df_report = pd.DataFrame(report).transpose()
#df_report.to_csv(os.path.join(os.getcwd(), f'clsreptwohidden.csv'), float_format='%.4f')

In [ ]:
# Comptue the non normalized confusion matrix
import seaborn as sns
cm = confusion_matrix(y_true_cat, y_pred_cat)
fig, ax = plt.subplots(figsize=(15,15))
heatmap = sns.heatmap(cm, fmt='g', cmap='Blues', annot=True, ax = ax)
ax.set_xlabel('Predicted class')
ax.set_ylabel('True class')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(target_names)
ax.yaxis.set_ticklabels(target_names)
# Save the heatmap to file
#heatmapfig = heatmap.get_figure()
#heatmapfig.savefig(f'confmat.png')